In [1]:
import time
import numpy as np
import pandas as pd
import datetime

from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# Controlling the webdriver  
Firstly, we want to access the webpage doctolib. To do that we have to create a connector with Selenium and the launch the driver

In [2]:

tempsAttente = 3    #time before a timeout in Webdriverwait.until()

class Driver:

    def __init__(self):
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument("--disable-blink-features")
        options.add_argument("disable-popup-blocking")
        options.add_argument("disable-notifications")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        options.add_argument("--incognito")
        # prefs = {"profile.default_content_settings.geolocation" : "2"}
        # options.add_experimental_option("prefs",prefs)
        self.driver = webdriver.Chrome(ChromeDriverManager().install())


    #Lancer google chrome avec la page doctolib

    def connect(self, url):
        self.driver.get(url)
        # TabsInteraction.TabsCreation(self.driver, "https://www.doctolib.fr", "")
        # TabsInteraction.switchToTab(self.driver, 0)
        print("Connecting to", url)
        time.sleep(2)
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//button[@id="didomi-notice-disagree-button"]')))
        self.driver.find_element(By.XPATH,'//button[@id="didomi-notice-disagree-button"]').click()


    #This function lets us get all the doctors for a certain specialization and city
    def getDoctors(self, specialization, city):

        doctors = []

        try :
            try :
                #writing the specialization
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
                self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
                self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(specialization)
                time.sleep(1)
            except :
                print("Pb avec la ligne 1")
            try :
                #writing the city
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]')))
                self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').clear()
                self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(city)
                time.sleep(1)
                #We need to enter the city to register it
                self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-place-input"]').send_keys(Keys.ENTER)
            except :
                print("Pb avec la ligne 2")
            try :
                #find the search button and click
                WebDriverWait(self.driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//span[text()='Rechercher']")))
                self.driver.find_element(By.XPATH, "//span[text()='Rechercher']").click()
            except :
                print("Pb avec la ligne 3")
                
            print('Searching results for :', specialization, 'in', city, 'at', datetime.datetime.now())
            time.sleep(2)

            try :
                WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]')))
                print(1)
                #number of doctors in city
                nbr_result = self.driver.find_element(By.XPATH, '//h4[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150 total-number-of-results dl-margin-r-8"]').text.split(" ")[0]
                
                print(2)
                nbr_result = int(nbr_result.split(" ")[0])
                print(nbr_result)
                # print("Il y a " + str(nbr_result//20)+"*20 + "+str(nbr_result%20)+" résultats !")
                    
                #nbr of doctors we want :
                if nbr_result<=100 :
                    pages = nbr_result//20
                else :
                    pages = int(0.2*int(nbr_result)) //20
                pages+=1
                print("il ya "+str(pages)+ "pages")
                
            except :
                print("Pb comptage pages")


            for i in range(1,pages):
                print("Nouvelle page")
                self.driver.execute_script("window.scrollTo(0, 6300);")
                
                
                WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-search-result"]')))
                docs = self.driver.find_elements(By.XPATH, '//div[@class="dl-search-result"]')
                
                
                for doctor in docs:
                    #first we get the "easy" features
                    name = doctor.find_element(By.XPATH, './/h3[@data-test="dl-search-result-name"]').text
                    city = city
                    specialization = specialization

                    #the harder ones
                    try:
                        WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]')))
                        first_dispo = doctor.find_element(By.XPATH, './/div[@class="Tappable-inactive availabilities-slot"]').get_attribute('aria-label')
                    except:
                        try:
                            WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/span[@class="dl-button-label"]')))
                            first_dispo = doctor.find_elements(By.XPATH, './/span[@class="dl-button-label"]')[1].text.split(" ")[3:5]
                        except:
                            try:
                                WebDriverWait(self.driver, 1).until(EC.visibility_of_element_located((By.XPATH, './/div[@class="dl-text dl-text-body dl-text-bold dl-text-s dl-text-neutral-150"]')))
                                first_dispo = 'pas de disponibilité'
                            except:
                                first_dispo = 'null'    #means that we didn't manage to find the date
                            

                    print(name, first_dispo)
                    fee = ""
                    doctors.append(Doctor(name, city, specialization, first_dispo, fee))
                
                
                try :
                    button = self.driver.find_element(By.XPATH, "//a[text()='Suivant']")
                    self.connect("https://www.doctolib.fr/"+specialization+"/"+city+"?page="+str(i+1))
                
                except : 
                    pass
                            

            
        
        except :
            print('Could not get the doctors for ' + specialization + ' in ' + city)

        return doctors


    

    #This function lets us get the fees for all doctors
    def getFees(self, name):
        self.driver.find_element(By.XPATH, '//div[@class="logo-doctolib logo-doctolib-white"]').click()

        #writing the doctor name
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').clear()
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input"]').send_keys(name)
        #We need to enter the city to register it
        #TODO : add the city
        WebDriverWait(self.driver, tempsAttente).until(EC.visibility_of_element_located((By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]')))
        self.driver.find_element(By.XPATH, '//input[@class="searchbar-input searchbar-query-input searchbar-query-input-opened"]').send_keys(Keys.ENTER)

        try:
            WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-fee-tag"]')))
            fee = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-fee-tag"]').text
            fee = str(int(fee.split(" ")[0]))
        except:
            try :
                WebDriverWait(self.driver, 0.5).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="dl-profile-text"]//div[1]')))
                first_sub_div = self.driver.find_element(By.XPATH, '//div[@class="dl-profile-text"]//div[1]')
                fee = first_sub_div.text
            except:
                fee = 'null'        

        return fee
            
        


We created a Doctor class to manipulate more easily the doctors features

In [3]:
class Doctor():
    def __init__(self, name, city, specialization, first_dispo, fee):
        self.name = name
        self.city = city
        self.specialization = specialization
        self.first_dispo = first_dispo
        self.fee = fee

    def features(self):
        return(self.name, self.city, self.specialization, self.first_dispo, self.fee)

These are the cities and specializations we looked at first :  
We decided to include the biggest specializations and some small, medium and big cities (30 in total)

In [4]:
cities = ['Poitiers', 'Rochefourchat', 'Majastres', 'Aulan', 'Rouen', 'Lorient', 'Niort', 'Valence', 'Quimper', 'Paris', 'Lyon', 'Marseille', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille', 'Poitiers', 'Dunkerque', 'Pau', 'Mulhouse', 'Lorient', 'Le Havre', 'Annecy', 'Troyes', 'La Rochelle', 'Amiens']
specializations = ['medecin-generaliste', 'dentiste', 'ophtalmologue', 'dermatologue', 'cardiologue', 'rhumatologue']


## Scrapping  
Here we did the scrapping to get all the doctors, we got their name, city, specialization and next appointment 

In [5]:
connector = Driver()
connector.connect("https://www.doctolib.fr")
doctors = []

#Get all the doctors names, cities, specializations and their next possible reservation

for city in cities:
    for spec in specializations:
        doctors = doctors + connector.getDoctors(spec, city)
        docs = []
        
        for doc in doctors:
            docs.append(doc.features())
        pd.DataFrame(docs, columns=['Name', 'City', 'Specialization', 'Waiting Time', 'Fees']).to_csv("doctors_new.csv")

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_8116/104985245.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr
Searching results for : medecin-generaliste in annecy at 2023-01-14 20:37:52.594353
1
2
357
il ya 4pages
Nouvelle page
Dr Linda DEZISSERT pas de disponibilité
Dr Anne-Nadine Coissard ['14', 'février']
Dr Maxime Duliege mar. 17 janv. 09:30
Dr Veronique LUKSIC lun. 16 janv. 11:00
Dr Patrick Chemla ['20', 'février']
Dr Philippe COLLET. mar. 17 janv. 12:15
Dr Pauline MILLON pas de disponibilité
Dr Elsa Jordan mar. 17 janv. 09:30
Dr Alain DEGRAMONT ['24', 'janvier']
Dr Augustin Decorde ['23', 'février']
Dr Fanny Perreau pas de disponibilité
Dr Pauline GOLDENBERG mar. 17 janv. 11:00
Dr Johanna BATAILLARD ['13', 'février']
Dr Emmanuelle PERRIN pas de disponibilité
Dr Lucie IMBERT pas de disponibilité
Dr Virginie Corbel pas de disponibilité
Dr Fabienne BINET pas de disponibilité
Dr Eva MARTIN pas de disponibilité
Dr Patricia BAVIANO pas de disponibilité
Dr Caroline PALANQUE pas de disponibilité
Connecting to https://www.doctolib.fr/medecin-generaliste/anne

## Scrapping of the fees
We then had to scrap the fees for each doctor by accessing their doctolib page

In [24]:

connector = Driver()
connector.connect("https://www.doctolib.fr")

doctors = pd.read_csv('doctors.csv')
doctors.pop(doctors.columns[0])


for i in range(len(doctors)):
    fee = connector.getFees(doctors['Name'][i])
    doctors['Fees'][i] = fee

    if i%50 == 0 :                             #we added in line in case selenium bugs, so that we do not lose all the progress made
        doctors.to_csv("doctors_fees.csv")
        print("Index : " + str(i))

    print(i,doctors['Name'][i], fee)


/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_1099/2064148729.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr


/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_1099/2355532568.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doctors['Fees'][i] = fee


Index : 0
0 Centre Médical de Soins Immédiats 86 Centre Médical de Soins Immédiats 86
1 Rue Henri le Châtelier, 86000 Poitiers
1 Marc BAHUET 25
2 Nathalie Dumureau Peltier 25
3 Dr Violaine Ricoulleau 25
4 Dr Alain SHEIKHALISHAHI 25
5 Dr Karine Brunet 25
6 Dr Cécile HUMMEL 25
7 Dr David CHAMBENOIT 60
8 Dr Airelle PLANQUES 25
9 Christophe INGRAND 25
10 Dr Angèle DEMION 50
11 Dr Elsa CANTOS 25
12 Laura BORTHOMIEU 25
13 Dr Céline BRECHON-GIRAUD Conventionné secteur 1
14 Dr Céline BRECHON-GIRAUD Conventionné secteur 1


KeyboardInterrupt: 

In [8]:
doctors.to_csv("doctors_fees.csv")

In [7]:
count = 1991
doctors = pd.read_csv('doctors_fees.csv')

In [8]:
while count < len(doctors):
    connector = Driver()
    connector.connect("https://www.doctolib.fr")

    try :
        for i in range(count, len(doctors)):
            count = i
            fee = connector.getFees(doctors['Name'][i])
            doctors['Fees'][i] = fee

            if i%50 == 0 :                             #we added this line in case selenium bugs, so that we do not lose all the progress made
                doctors.to_csv("doctors_fees.csv")
                print("Index : " + str(i))

            print(i,doctors['Name'][i], fee)
            
            
    except :
        doctors.to_csv("doctors_fees.csv")
        count = count + 1                           #sometimes it bugs with a doctor so we need to discard it
        print("Error at : " + str(count - 1))

/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_1160/937219280.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


Connecting to https://www.doctolib.fr


/var/folders/bm/t8hxv1ms6s30y8czfw0pmfk00000gn/T/ipykernel_1160/128489326.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doctors['Fees'][i] = fee


1991 Dr Annelle Deyehe 23
1992 Issa NAJM 31 Rue de Caumartin, 75009 Paris
1993 Dr Nathalie AZOULAY 23
1994 Hôpital de la Pitié-Salpêtrière - AP-HP Établissement conventionné
1995 Dr Mathilde CUEVAS 23
1996 Dr Alejandra HAAS 90
1997 Dr Patrick PISSIS 90
1998 Dr Hermine Arissian 23
1999 Dr Nicolas Bavoux Cabinet d'Implantologie et d'esthétique dentaires Carré Bavoux
3 Rue Tellier Frères, 78750 Mareil-Marly
Error at : 2000
Connecting to https://www.doctolib.fr


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.87)
Stacktrace:
0   chromedriver                        0x00000001029670fc chromedriver + 4223228
1   chromedriver                        0x00000001028ef284 chromedriver + 3732100
2   chromedriver                        0x00000001025a35c8 chromedriver + 275912
3   chromedriver                        0x000000010257d7b0 chromedriver + 120752
4   chromedriver                        0x00000001026040f8 chromedriver + 671992
5   chromedriver                        0x0000000102616b18 chromedriver + 748312
6   chromedriver                        0x00000001025d1748 chromedriver + 464712
7   chromedriver                        0x00000001025d27f0 chromedriver + 468976
8   chromedriver                        0x0000000102937e08 chromedriver + 4029960
9   chromedriver                        0x000000010293b698 chromedriver + 4044440
10  chromedriver   

In [31]:
doctors.to_csv("doctors_fees.csv")


In [7]:
doctors['Fees']

0       Centre Médical de Soins Immédiats 86\n1 Rue He...
1                                                      25
2                                                      25
3                                                      25
4                                                      25
                              ...                        
6873                                                  NaN
6874                                                  NaN
6875                                                  NaN
6876                                                  NaN
6877                                                  NaN
Name: Fees, Length: 6878, dtype: object